In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-aug-2022/sample_submission.csv
/kaggle/input/tabular-playground-series-aug-2022/train.csv
/kaggle/input/tabular-playground-series-aug-2022/test.csv


In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings('ignore')

In [3]:
train_path = '/kaggle/input/tabular-playground-series-aug-2022/train.csv'
test_path = '/kaggle/input/tabular-playground-series-aug-2022/test.csv'
submission_path = '/kaggle/input/tabular-playground-series-aug-2022/sample_submission.csv'

In [4]:
train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)
sample_sub = pd.read_csv(submission_path)

In [5]:
train_data['failure'].value_counts()

0    20921
1     5649
Name: failure, dtype: int64

In [6]:
print('Train data missing value is = {} %'.format(100 * train_data.isna().sum().sum() / (len(train_data)*24)))
print('Test data missing value is  = {} %'.format(100 * test_data.isna().sum().sum() / (len(test_data)*24)))

Train data missing value is = 3.1791807803286916 %
Test data missing value is  = 3.1506217408744486 %


In [7]:
train_data.fillna(train_data.mean(), inplace=True)
test_data.fillna(train_data.mean(), inplace=True)

In [8]:
test_data['failure'] = -1
data = pd.concat([train_data, test_data]).reset_index(drop=True)

In [9]:
cat_columns = ['product_code', 'attribute_0', 'attribute_1']
num_columns = [col for col in train_data.columns if col not in ['failure', 'id'] + cat_columns]
feature_columns = cat_columns + num_columns
target = 'failure'

In [10]:
for col in cat_columns:
    le = LabelEncoder()
    le.fit(data[col])
    train_data[col] = le.transform(train_data[col])
    test_data[col] = le.transform(test_data[col])

In [11]:
train_data.head()

,id,product_code,loading,attribute_0,attribute_1,attribute_2,attribute_3,measurement_0,measurement_1,measurement_2,...,measurement_9,measurement_10,measurement_11,measurement_12,measurement_13,measurement_14,measurement_15,measurement_16,measurement_17,failure
0,0,0,80.10,1,3,9,5,7,8,4,...,10.672,15.859,17.594000,15.193,15.029,16.048444,13.034,14.684,764.100,0
1,1,0,84.89,1,3,9,5,14,3,3,...,12.448,17.947,17.915000,11.755,14.732,15.425000,14.395,15.631,682.057,0
2,2,0,82.43,1,3,9,5,12,1,5,...,12.715,15.607,19.172085,13.798,16.711,18.631000,14.094,17.946,663.376,0
3,3,0,101.07,1,3,9,5,13,2,6,...,12.471,16.346,18.377000,10.020,15.250,15.562000,16.154,17.172,826.282,0
4,4,0,188.06,1,3,9,5,9,2,8,...,10.337,17.082,19.932000,12.428,16.182,12.760000,13.153,16.412,579.885,0


In [12]:
model = LogisticRegression(max_iter = 500, C=0.0001, penalty='l2', solver='newton-cg')

In [13]:
train = train_data[feature_columns]
label = train_data[target]
test = test_data[feature_columns]

In [14]:
kfold = 5
oof_preds = np.zeros((train_data.shape[0]))
test_preds = np.zeros(test_data.shape[0])
skf = StratifiedKFold(n_splits=kfold)

for k, (train_index, test_index) in enumerate(skf.split(train, label)):
    x_train, x_test = train.iloc[train_index, :], train.iloc[test_index, :]
    y_train, y_test = label.iloc[train_index], label.iloc[test_index]
    
    model.fit(x_train,y_train)
    
    y_pred = model.predict_proba(x_test)[:,1]
    oof_preds[test_index] = y_pred.ravel()
    auc = roc_auc_score(y_test,y_pred)
    print("KFold %d, the best auc is %.4f" % (k, auc))
    test_fold_preds = model.predict_proba(test)[:, 1]
    test_preds += test_fold_preds.ravel()
print("Overall AUC={}".format(roc_auc_score(label, oof_preds)))

KFold 0, the best auc is 0.5976
KFold 1, the best auc is 0.5864
KFold 2, the best auc is 0.5879
KFold 3, the best auc is 0.5976
KFold 4, the best auc is 0.5810
Overall AUC=0.5885825246089892


In [15]:
test_preds /= kfold
test_preds = pd.Series(test_preds, name='failure')
sample_sub['failure'] = test_preds
submission = sample_sub.copy()
submission.to_csv('submission.csv',index= False)

In [16]:
submission.head()

,id,failure
0,26570,0.199871
1,26571,0.169476
2,26572,0.185729
3,26573,0.188955
4,26574,0.351334
